In [ ]:
import json
import glob
import pickle
import ast
from collections import defaultdict
import pandas as pd
from tqdm import tqdm
from multiprocessing.pool import ThreadPool as Pool

In [ ]:
districts = ['agra', 'aligarh', 'varanasi', 'lucknow', 'ghaziabad', 'ambedkar_nagar', 'bahraich', 'azamgarh',
             'allahabad', 'balrampur', 'auraiya', 'barabanki', 'banda', 'bagpat', 'bhadohi', 'ballia', 'bijnor',
             'basti', 'bareilly', 'bulandshahar', 'chitrakoot', 'deoria', 'budaun', 'etah', 'etawah', 'farrukhabad',
             'faizabad', 'fatehpur', 'firozabad', 'gautam_buddha_nagar', 'ghazipur', 'hapur', 'gonda', 'hardoi',
             'hamirpur_up', 'jaunpur', 'gorakhpur', 'jalaun', 'jyotiba_phule_nagar', 'jhansi', 'hathras', 'kanpur_dehat',
             'kanpur_nagar', 'kannauj', 'kanshiramnagar', 'kheri', 'kaushambi', 'kushinagar', 'lalitpur', 'maharajganj',
             'mainpuri', 'meerut', 'mahoba', 'mirzapur', 'mathura', 'moradabad', 'muzaffarnagar', 'pratapgarhdistrict', 'pilibhit',
             'rampur', 'raebareli', 'mau', 'saharanpur', 'sant_kabir_nagar', 'shahjahanpur', 'shravasti', 'siddharthnagar', 'sitapur', 'unnao',
             'sultanpur', 'sonbhadra']

In [ ]:
assert len(districts) == 71
HOME = "../all_bail_cases_pickles/"
def file(district):
  return f"{HOME}/{district}/full_data_after_simple_NER_division.json"

In [ ]:
total_words = 0
total_docs = 0

for district in tqdm(districts):
  with open(file(district), 'r') as f:
    data = json.load(f)
#   print(district)
  for court in data.keys():
    df = pd.DataFrame(data[court]['processed']).T
    for idx, i in df.iterrows():
        text = i['header'] + " ".join(i['body']) + i['result']
        total_words += len(text.split())
        total_docs += 1
    for i in data[court]['valid']:
        text = data[court]['valid'][i]
        total_words += len(text.split())
        total_docs += 1

In [ ]:
len(text.split())

In [ ]:
total_words

In [ ]:
total_docs

In [ ]:
total_words/total_docs

In [ ]:
len(text_per_district)

In [ ]:
sent_count = 0
for dst in text_per_district:
    sent_count += text_per_district[dst].count("।")

In [ ]:
pickle.dump(text_per_district, open("textperdistrict", "wb"))

In [ ]:
text_per_district = pickle.load(open("textperdistrict", "rb"))

In [7]:
len(text_per_district)

71

time: 8.75 ms (started: 2021-11-02 11:58:05 +05:30)


In [ ]:
from lexical_diversity import lex_div as ld

In [ ]:
split_text = {}

for i in tqdm(text_per_district):
    text = " ".join(text_per_district[i].split("।"))
    text = " ".join(text.split("|"))
    flt = ld.flemmatize(text)
    split_text[i] = flt
#     val = ld.ttr(flt)
#     print(i, val)
    
    # USE punctuations for splitting also; Rather replace all with space
#     text = " ".join(text_per_district[i].split("।"))
#     text = " ".join(text.split("|"))
#     text = text.split()
#     split_text[i] = text

In [ ]:
del text_per_district
# del split_text

In [ ]:
term_count = {}
for i in tqdm(split_text):
    text = split_text[i]
    for j in text:
        # IGNORE words with more than 75% digits; or english letters
        
        if j in term_count:
            if i in term_count[j]:
                term_count[j][i] += 1
            else:
                term_count[j][i] = 1
        else:
            term_count[j] = {i: 1}       

In [ ]:
len(term_count)

In [ ]:
pickle.dump(term_count, open("term_count", "wb"))

In [ ]:
term_count = pickle.load(open("term_count", "rb"))

In [ ]:
len(term_count)

In [ ]:
for i in term_count:
    if len(term_count[i]) == 71:
        cnt = 0
        for j in term_count[i]:
            cnt += term_count[i][j]
        if cnt > 10000:
            continue
#         print(i, cnt)
#         input()

In [ ]:
stopwords = [
  "मैं", "मुझको", "मेरा", "अपने आप को", "हमने", "हमारा", "अपना", "हम", "आप", "आपका", "तुम्हारा", "अपने आप", "स्वयं", "वह", "इसे", 
"उसके", "खुद को", "कि वह", "उसकी", "उसका", "खुद ही", "यह", "इसके", "उन्होने", "अपने", "क्या", "जो", "किसे", "किसको", "कि", "ये",
 "हूँ", "होता है", "रहे", "थी", "थे", "होना", "गया", "किया जा रहा है", "किया है", "है", "पडा", "होने", "करना", "करता है", "किया", "रही", 
 "एक", "लेकिन", "अगर", "या", "क्यूंकि", "जैसा", "जब तक", "जबकि", "की", "पर", "द्वारा", "के लिए", "साथ", "के बारे में", "खिलाफ", 
 "बीच", "में", "के माध्यम से", "दौरान", "से पहले", "के बाद", "ऊपर", "नीचे", "को", "से", "तक", "से नीचे", "करने में", "निकल", "बंद", "से अधिक",
  "तहत", "दुबारा", "आगे", "फिर", "एक बार", "यहाँ", "वहाँ", "कब", "कहाँ", "क्यों", "कैसे", "सारे", "किसी", "दोनो", "प्रत्येक", 
 "ज्यादा", "अधिकांश", "अन्य", "में कुछ", "ऐसा", "में कोई", "मात्र", "खुद", "समान", "इसलिए", "बहुत", "सकता", "जायेंगे", "जरा", "चाहिए", 
 "अभी", "और", "कर दिया", "रखें", "का", "हैं", "इस", "होता", "करने", "ने", "बनी", "तो", "ही", "हो", "इसका", "था", "हुआ", "वाले", 
 "बाद", "लिए", "सकते", "इसमें", "दो", "वे", "करते", "कहा", "वर्ग", "कई", "करें", "होती", "अपनी", "उनके", "यदि", "हुई", "जा", "कहते", 
 "जब", "होते", "कोई", "हुए", "व", "जैसे", "सभी", "करता", "उनकी", "तरह", "उस", "आदि", "इसकी", "उनका", "इसी", "पे", "तथा", "भी",
 "परंतु", "इन", "कम", "दूर", "पूरे", "गये", "तुम", "मै", "यहां", "हुये", "कभी", "अथवा", "गयी", "प्रति", "जाता", "इन्हें", "गई", "अब",
 "जिसमें", "लिया", "बड़ा", "जाती", "तब", "उसे", "जाते", "लेकर", "बड़े", "दूसरे", "जाने", "बाहर", "स्थान", "उन्हें ", "गए", "ऐसे", 
 "जिससे", "समय", "दोनों", "किए", "रहती", "इनके", "इनका", "इनकी", "सकती", "आज", "कल", "जिन्हें", "जिन्हों", "तिन्हें", "तिन्हों", 
 "किन्हों", "किन्हें", "इत्यादि", "इन्हों", "उन्हों", "बिलकुल", "निहायत", "इन्हीं", "उन्हीं", "जितना", "दूसरा", "कितना", "साबुत", "वग़ैरह", 
 "कौनसा", "लिये", "दिया", "जिसे", "तिसे", "काफ़ी", "पहले", "बाला", "मानो", "अंदर", "भीतर", "पूरा", "सारा", "उनको", "वहीं", "जहाँ",
 "जीधर", "के", "एवं", "कुछ", "कुल", "रहा", "जिस", "जिन", "तिस", "तिन", "कौन", "किस", "संग", "यही", "बही", "उसी", "मगर",
 "कर", "मे", "एस", "उन", "सो", "अत", 'के', 'है।', '/', 'था।', 
 

 # varanasi
 "न्यायालय", "सत्र", "न्यायाधीश", "जमानत", "प्रार्थना", "पत्र", "अभियुकतगण", "अभियुक्त", "जिला", "शासकीय", "तर्को", "नहीं", "आवेदक", "प्रार्थी", 
 "प्रार्थनापत्र", "विद्वान", "अधिवक्ता", "Bail", "Application", "वाराणसी", "प्रथम", "सं०ः", "तर्क", "अलावा", "अभियुक्तगण", "बनाम", "संख्या", "धारा",
 "बनाम", "राज्य", "दिन", "विवाद", "करती", "यानी", "भा०दं०्सं०", "आदेश", "दिनांक", "होकर", "जिसके", "हेतु", "तथ्यों", "कानूनी", "District", 
 "Court", "वालों", "लोगों", "मुकदमा", "पत्रावली", "करेंगे", 'प्रस्तुत', 'अपराध', 'अभियोजन', 'पुत्र', 'रिपोर्ट', 'सूचना', 'भादंसं', 'व्यक्ति', 
 'पंजीकृत', 'मामले', 'बताया', 'मुअसं', 'रूपये', 'पुलिस', 'करायी', 'अन्तर्गत', 'अधिनियम', 'उपरोक्त', 'अग्रिम', 'पीड़िता', 'कराया', 'अवलोकन', 'पूर्व', 
 'विशेष', 'आपराधिक', 'Location', 'Sessions', 'मजिस्ट्रेट', 'प्रदेश', 'उत्तर', 'निवासी', 'बरामद', 'मैंने', 'वादिनी', 'भादसं', 'बरामदगी', 'निरस्त',
 'जिलावाराणसी', 'संबंध', 'अंतर्गत', 'प्रकार', 'इतिहास', 'बरामदगी', 'सहायक', 'साक्षी', 'न्यायिक', 'माननीय', 'पाण्डेय', 'संक्षेप', 
 'फौजदारी', 'कथानक', 'Reason', 'Document', 'प्रपत्रों', 'उपलब्ध', 'अभियुक्ता', 'विवेचना', 'सम्बन्ध', 'विरूद्ध', 'विरोध', 'ग्राम', 'मुकदमें', 'sessssessssneeeess'

 # agra
 'कारित', 'मिलकर', 'याचना', 'श्रीमती', 'अभियोजक', 'अधिकारी', 'निरूद्ध', 'याचना', 'दिनाक', 'विपक्षी', 'निर्दोष', 'प्राथमिकी', 'State', 'सम्बन्धित',
  'आधारों', 'जायेगा', 'अभियोजक', 'आवेदन', 'अभियोजक', 'प्रदान', 'अपराधी'
]

stopwords_list2 = ['', 'अर्थात', 'कुछ', 'तेरी', 'साबुत', 'अपनि', 'हूं', 'काफि', 'यिह', 'जा' ,'दे', 'देकर' ,'रह', 'कह' , 
                   'कर' , 'कहा', 'बात' , 'जिन्हों', 'किर', 'कोई', 'हे', 'कोन', 'रहा', 'सब', 'सो', 'तक', 'इंहें', 'इसकि', 
                   'अपनी', 'दबारा', 'सभि', 'होते', 'भीतर', 'निचे', 'घर', 'उन्हें', 'उन्ह' , 'मेरे' , 'था', 'व', 'इसमें', 'उसी', 
                   'बिलकुल', 'होति', 'गया', 'सकता', 'अपना', 'लिये', 'उसका', 'पर', 'दवारा', 'गए', 'है', 'कितना', 'भि', 'लिए', 'वुह ',
                   'ना', 'किसि', 'परन्तु', 'किन्हें', 'बहुत', 'भी', 'तुम्हारे', 'निहायत', 'उन्हीं', 'वहिं', 'हैं', 'उन्हों', 'इतयादि','यहाँ', 'तब', 'पूरा', 
                   'क्योंकि', 'कौनसा', 'आप', 'हुअ', 'ऐसे', 'एस', 'कारण', 'अप', 'पहले', 'तुम', 'जेसा', 'तिस', 'लेकिन', 'कहते', 'मगर', 'करता', 'संग', 
                   'सभी', 'जीधर', 'किंहों', 'हि', 'द्वारा', 'हुआ', 'तू', 'जिंहें', 'उसने', 'पास', 'वहां', 'वह', 'किंहें', 'इंहों', 'मुझ', 'कुल', 'तिंहों', 'का',
                   'मेरी', 'तेरे', 'उनके', 'क्या', 'जहाँ', 'काफ़ी', 'वर्ग', 'वरग','बही', 'ये', 'जिस', 'इसि', 'हुई', 'साम्हने', 'नहिं', 'जैसे', 'वहीं', 'दिया',
                   'अभी', 'यहि', 'वग़ैरह', 'उनकि', 'न', 'जा','बनि', 'हें', 'यिह ', 'उन', 'को', 'तिन्हों', 'उन्होंने', 'तुझे', 'उसे', 'होने', 'इन्हीं', 'थे',
                   'उंहिं', 'अपने', 'में', 'फिर','यही', 'नीचे', 'होती', 'तिसे', 'हम', 'यदि', 'सारा', 'कर', 'सकते', 'कोइ', 'और', 'जिंहों', 'तिंहें', 'दूसरे', 
                   'जब', 'रहे','अत', 'मानो', 'जिन', 'बाद', 'उनका', 'किया', 'या', 'उनकी', 'कौन', 'ऐसा', 'सबसे', 'अनुसार', 'दुसरे', 'इन', 'अदि','जिसे',
                   'उसकी', 'इत्यादि', 'करना', 'यहां', 'हुए', 'तेरा', 'आदि', 'पर  ', 'वाले', 'कहता', 'किन्हों', 'किसे', 'जिन्हें', 'मे','होता', 'करने', 'साभ',
                   'अभि', 'उसको', 'कई', 'बनी', 'के', 'इन्हें', 'वहाँ', 'कोनसा', 'कइ', 'इनका', 'थि', 'बाला','ऱ्वासा', 'हो', 'उंहें', 'दुसरा', 'वे', 'भितर',
                   'जेसे', 'एवं', 'अंदर', 'दो', 'साथ', 'करें', 'जिधर', 'तरह', 'उसि', 'इस', 'एसे', 'तिन', 'नहीं', 'से','न','उनको', 'किस', 'किसी', 'इसी',
                   'मैं', 'यह', 'हुइ', 'ले', 'कि', 'की', 'इसलिये', 'रवासा', 'ने', 'जैसा', 'वह ', 'तिन्हें', 'वुह', 'उस', 'उंहों', 'वगेरह', 'उसके', 'मुझे', 'करते',
                   'जितना', 'जहां', 'इन्हों', 'इसके', 'होना', 'इसका', 'इंहिं', 'एक', 'जो', 'पे', 'ही', 'तो', 'थी', 'रखें', 'इसे', 'इन ', 'के', 'बहि', 'पुरा', 
                   'ओर', 'इसकी']

stopwords_list3 = [
                   "अत", "अपना", "अपनी", "अपने", "अभी", "अंदर", "आदि", "आप", "इत्यादि", "इन ", "इनका", "इन्हीं", "इन्हें", "इन्हों", "इस", "इसका", 
"इसकी", "इसके", "इसमें", "इसी", "इसे", "उन", "उनका", "उनकी", "उनके", "उनको", "उन्हीं", "उन्हें", "उन्हों", "उस", "उसके", "उसी",
"उसे", "एक", "एवं", "एस", "ऐसे", "और", "कई", "कर", "करता", "करते", "करना", "करने", "करें", "कहते", "कहा", "का", "काफ़ी", "कि", 
"कितना", "किन्हें", "किन्हों", "किया", "किर", "किस", "किसी", "किसे", "की", "कुछ", "कुल", "के", "को", "कोई", "कौन", "कौनसा", "गया", 
"घर", "जब", "जहाँ", "जा", "जितना", "जिन", "जिन्हें", "जिन्हों", "जिस", "जिसे", "जीधर", "जैसा", "जैसे", "जो", "तक", "तब", "तरह", 
"तिन", "तिन्हें", "तिन्हों", "तिस", "तिसे", "तो", "था", "थी", "थे", "दबारा", "दिया", "दुसरा", "दूसरे", "दो", "द्वारा", "न", "नके", "नहीं", 
"ना", "निहायत", "नीचे", "ने", "पर", "पहले", "पूरा", "पे", "फिर", "बनी", "बही", "बहुत", "बाद", "बाला", "बिलकुल", "भी", "भीतर", "मगर", 
"मानो", "मे", "में", "यदि", "यह", "यहाँ", "यही", "या", "यिह", "ये", "रखें", "रहा", "रहे", "ऱ्वासा", "लिए", "लिये", "लेकिन", "व", "वग़ैरह", 
"वर्ग", "वह", "वहाँ", "वहीं", "वाले", "वुह", "वे", "सकता", "सकते", "सबसे", "सभी", "साथ", "साबुत", "साभ", "सारा", "से", "सो", "संग", 
"ही", "हुआ", "हुई", "हुए", "है", "हैं", "हो", "होता", "होती", "होते", "होना", "होने", "अंदर", "अत", "अदि", "अप", "अपना", "अपनि", 
"अपनी", "अपने", "अभि", "अभी", "आदि", "आप", "इंहिं", "इंहें", "इंहों", "इतयादि", "इत्यादि", "इन", "इनका", "इन्हीं", "इन्हें", "इन्हों", "इस", 
"इसका", "इसकि", "इसकी", "इसके", "इसमें", "इसि", "इसी", "इसे", "उंहिं", "उंहें", "उंहों", "उन", "उनका", "उनकि", "उनकी", "उनके", 
"उनको", "उन्हीं", "उन्हें", "उन्हों", "उस", "उसके", "उसि", "उसी", "उसे", "एक", "एवं", "एस", "एसे", "ऐसे", "ओर", "और", "कइ", "कई", 
"कर", "करता", "करते", "करना", "करने", "करें", "कहते", "कहा", "का", "काफि", "काफ़ी", "कि", "किंहें", "किंहों", "कितना", "किन्हें", 
"किन्हों", "किया", "किर", "किस", "किसि", "किसी", "किसे", "की", "कुछ", "कुल", "के", "को", "कोइ", "कोई", "कोन", "कोनसा", "कौन", 
"कौनसा", "गया", "घर", "जब", "जहाँ", "जहां", "जा", "जिंहें", "जिंहों", "जितना", "जिधर", "जिन", "जिन्हें", "जिन्हों", "जिस", "जिसे", "जीधर", 
"जेसा", "जेसे", "जैसा", "जैसे", "जो", "तक", "तब", "तरह", "तिंहें", "तिंहों", "तिन", "तिन्हें", "तिन्हों", "तिस", "तिसे", "तो", "था", "थि", 
"थी", "थे", "दबारा", "दवारा", "दिया", "दुसरा", "दुसरे", "दूसरे", "दो", "द्वारा", "न", "नहिं", "नहीं", "ना", "निचे", "निहायत", "नीचे", "ने", 
"पर", "पहले", "पुरा", "पूरा", "पे", "फिर", "बनि", "बनी", "बहि", "बही", "बहुत", "बाद", "बाला", "बिलकुल", "भि", "भितर", "भी", "भीतर", 
"मगर", "मानो", "मे", "में", "यदि", "यह", "यहाँ", "यहां", "यहि", "यही", "या", "यिह", "ये", "रखें", "रवासा", "रहा", "रहे", "ऱ्वासा", "लिए", 
"लिये", "लेकिन", "व", "वगेरह", "वरग", "वर्ग", "वह", "वहाँ", "वहां", "वहिं", "वहीं", "वाले", "वुह", "वे", "वग़ैरह", "संग", "सकता", "सकते", 
"सबसे", "सभि", "सभी", "साथ", "साबुत", "साभ", "सारा", "से", "सो", "हि", "ही", "हुअ", "हुआ", "हुइ", "हुई", "हुए", "हे", "हें", "है", 
"हैं", "हो", "होता", "होति", "होती", "होते", "होना", "होने", "मैं", "मुझको", "मेरा", "अपने आप को", "हमने", "हमारा", "अपना", "हम", "आप", 
"आपका", "तुम्हारा", "अपने आप", "स्वयं", "वह", "इसे", "उसके", "खुद को", "कि वह", "उसकी", "उसका", "खुद ही", "यह", "इसके", "उन्होने", 
"अपने", "क्या", "जो", "किसे", "किसको", "कि", "ये", "हूँ", "होता है", "रहे", "थी", "थे", "होना", "गया", "किया जा रहा है", "किया है", "है", 
"पडा", "होने", "करना", "करता है", "किया", "रही", "एक", "लेकिन", "अगर", "या", "क्यूंकि", "जैसा", "जब तक", "जबकि", "की", "पर", 
"द्वारा", "के लिए", "साथ", "के बारे में", "खिलाफ", "बीच", "में", "के माध्यम से", "दौरान", "से पहले", "के बाद", "ऊपर", "नीचे", "को", "से", 
"तक", "से नीचे", "करने में", "निकल", "बंद", "से अधिक", "तहत", "दुबारा", "आगे", "फिर", "एक बार", "यहाँ", "वहाँ", "कब", "कहाँ", "क्यों", 
"कैसे", "सारे", "किसी", "दोनो", "प्रत्येक", "ज्यादा", "अधिकांश", "अन्य", "में कुछ", "ऐसा", "में कोई", "मात्र", "खुद", "समान", "इसलिए", 
"बहुत", "सकता", "जायेंगे", "जरा", "चाहिए", "अभी", "और", "कर दिया", "रखें", "का", "हैं", "इस", "होता", "करने", "ने", "बनी", "तो", "ही", 
"हो", "इसका", "था", "हुआ", "वाले", "बाद", "लिए", "सकते", "इसमें", "दो", "वे", "करते", "कहा", "वर्ग", "कई", "करें", "होती", "अपनी", 
"उनके", "यदि", "हुई", "जा", "कहते", "जब", "होते", "कोई", "हुए", "व", "जैसे", "सभी", "करता", "उनकी", "तरह", "उस", "आदि", "इसकी", 
"उनका", "इसी", "पे", "तथा", "भी", "परंतु", "इन", "कम", "दूर", "पूरे", "गये", "तुम", "मै", "यहां", "हुये", "कभी", "अथवा", "गयी", "प्रति", 
"जाता", "इन्हें", "गई", "अब", "जिसमें", "लिया", "बड़ा", "जाती", "तब", "उसे", "जाते", "लेकर", "बड़े", "दूसरे", "जाने", "बाहर", "स्थान", 
"उन्हें ", "गए", "ऐसे", "जिससे", "समय", "दोनों", "किए", "रहती", "इनके", "इनका", "इनकी", "सकती", "आज", "कल", "जिन्हें", "जिन्हों", 
"तिन्हें", "तिन्हों", "किन्हों", "किन्हें", "इत्यादि", "इन्हों", "उन्हों", "बिलकुल", "निहायत", "इन्हीं", "उन्हीं", "जितना", "दूसरा", "कितना", "साबुत", 
"वग़ैरह", "कौनसा", "लिये", "दिया", "जिसे", "तिसे", "काफ़ी", "पहले", "बाला", "मानो", "अंदर", "भीतर", "पूरा", "सारा", "उनको", "वहीं", "जहाँ", 
"जीधर", "के", "एवं", "कुछ", "कुल", "रहा", "जिस", "जिन", "तिस", "तिन", "कौन", "किस", "संग", "यही", "बही", "उसी", "मगर", "कर", 
"मे", "एस", "उन", "सो", "अत", "पर  ", "इन ", "वह ", "यिह ", "वुह ", "जिन्हें", "जिन्हों", "तिन्हें", "तिन्हों", "किन्हों", "किन्हें", 
"इत्यादि", "द्वारा", "इन्हें", "इन्हों", "उन्हों", "बिलकुल", "निहायत", "ऱ्वासा", "इन्हीं", "उन्हीं", "उन्हें", "इसमें", "जितना", "दुसरा", "कितना", 
"दबारा", "साबुत", "वग़ैरह", "दूसरे", "कौनसा", "लेकिन", "होता", "करने", "किया", "लिये", "अपने", "नहीं", "दिया", "इसका", "करना", "वाले", 
"सकते", "इसके", "सबसे", "होने", "करते", "बहुत", "वर्ग", "करें", "होती", "अपनी", "उनके", "कहते", "होते", "करता", "उनकी", "इसकी", 
"सकता", "रखें", "अपना", "उसके", "जिसे", "तिसे", "किसे", "किसी", "काफ़ी", "पहले", "नीचे", "बाला", "यहाँ", "जैसा", "जैसे", "मानो", "अंदर", 
"भीतर", "पूरा", "सारा", "होना", "उनको", "वहाँ", "वहीं", "जहाँ", "जीधर", "उनका", "इनका", "के", "हैं", "गया", "बनी", "एवं", "हुआ", "साथ", 
"बाद", "लिए", "कुछ", "कहा", "यदि", "हुई", "इसे", "हुए", "अभी", "सभी", "कुल", "रहा", "रहे", "इसी", "उसे", "जिस", "जिन", "तिस", 
"तिन", "कौन", "किस", "कोई", "ऐसे", "तरह", "किर", "साभ", "संग", "यही", "बही", "उसी", "फिर", "मगर", "का", "एक", "यह", "से", 
"को", "इस", "कि", "जो", "कर", "मे", "ने", "तो", "ही", "या", "हो", "था", "तक", "आप", "ये", "थे", "दो", "वे", "थी", "जा", "ना", 
"उस", "एस", "पे", "उन", "सो", "भी", "और", "घर", "तब", "जब", "अत", "व", "न"
]

stopwords.extend(stopwords_list2)
stopwords.extend(stopwords_list3)
stopwords = set(stopwords)

In [ ]:
for i in term_count:
    if len(term_count[i]) == 1:
        cnt = 0
        for j in term_count[i]:
            cnt += term_count[i][j]
        if cnt <100:
            continue
        count_letters = 0
        count_digits = 0
        ti = i
        i = i.replace("<फ़ोननंबर>", "")
        for j in i:
            if 'a' <= j <= 'z' or 'A' <= j <= 'Z':
                count_letters += 1
            if '0' <= j <= '9':
                count_digits += 1
                
        if len(i) == 0:
            continue
        if i in stopwords or ti in stopwords:
            continue
        if count_letters/len(i) >= 0.75 or count_digits/len(i) >= 0.75 or (count_letters + count_digits)/len(i) >= 0.75:
            continue
        print(ti, term_count[ti])
        
#         input()

In [ ]:
len(term_count)